**_test_data-update_data.ipynb_ - Update unit test data to include OpenCalais V.2 coding**

# TODO

- update test data:

    - // load existing fixtures.
    - re-code with OpenCalais v.2
    
        - add API key to config
        - code all Articles
        - remove API key from config
    
    - make sure we can generate network data from them.
    - if no one with single names, make it so a few have single names in Article_Source.
    - build out network data for a few different specs (one with single name, one without) and store in new table.
    - re-export fixtures (notes on how to do this?).

- make sure existing unit tests work with new data.
- make sure the Article_Data method `filter_article_persons()` works as I intend. To start, create tests in notebook against actual database, using full Article_Subject and Article_Author QuerySets, compare numbers to raw queries. Then, do the same against test database, use numbers to create unit tests.
- even lower level, make tests for the method to build person dictionaries, and the base lookup method.
- simple network data creation test - run with a few specs against test data, make sure I get the right size of output back for each.

# Setup

## Setup - Imports

In [3]:
# python base imports
import datetime
import json

# import six
import six

print( "packages imported at " + str( datetime.datetime.now() ) )

packages imported at 2022-05-24 01:21:28.208492


## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [5]:
%run ../../django_init.py

django initialized at 2022-05-24 01:21:37.437492


In [6]:
# django imports
from django.contrib.auth.models import User
from django.db.models import Max
from django.db.models import Min

# sourcenet imports
from context_text.shared.context_text_base import ContextTextBase

# context_analysis imports
from context_analysis.network.network_person_info import NetworkPersonInfo

# sourcenet imports
from context_text.models import Article
from context_text.models import Article_Author
from context_text.models import Article_Data
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.models import Person

# article coding
from context_text.article_coding.article_coder import ArticleCoder
#from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.article_coding.open_calais_v2.open_calais_v2_api_response import OpenCalaisV2ApiResponse

# import class that actually processes requests for outputting networks.
from context_text.export.network_output import NetworkOutput

# context_text shared
from context_text.shared.context_text_base import ContextTextBase

print( "django model packages imported at " + str( datetime.datetime.now() ) )

django model packages imported at 2022-05-24 01:21:39.152368


## Setup - shared variables

In [7]:
# get ArticleCoding instance.
#article_coding = ArticleCoding()

# automated coding user
automated_coder = ArticleCoder.get_automated_coding_user()

# newspapers for Grand Rapids Press and Detroit News.
grand_rapids_press = Newspaper.objects.get( newsbank_code = "GRPB" )
detroit_news = Newspaper.objects.get( newsbank_code = "DTNB" )

# OpenCalais v2 coder type
ocv2_coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION

# Understand existing data

In [8]:
# how many articles?
article_qs = Article.objects.all()
print( "Article count: {}".format( article_qs.count() ) )

Article count: 46
